### Hierarchical VI

In [1]:
from ihrl.hvi import val_true
from ihrl.taxicab import TaxiMDP, Root, taxi_state

layout_str = """
A--B
----
----
C--D 
"""
mdp = TaxiMDP(layout_str)
init_subtask = Root(mdp)
init_state = taxi_state(0, 0, mdp.width-1, mdp.height-1)
val = val_true(init_subtask, init_state, max_horizon=10)

In [2]:
val

21.0